In [0]:
#!git clone https://github.com/vivek7mehta/Kaggle.git
#!mv ./Kaggle/av-innoplexus/data ./
#!unzip test_XEV14AD.zip

In [1]:
import pandas as pd
import spacy
from sklearn.model_selection import train_test_split
import numpy as np
from keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt
from eval_metric import compute_metrics
import eval_metric
import tensorflow as tf

Using TensorFlow backend.


In [0]:
def chunks(l, n):
    lsts = []
    for i in range(0, len(l), n):
        lsts.append(l[i:i + n])
    return lsts
  

In [0]:
train = pd.read_csv('train.csv')
train_copy = pd.read_csv('train.csv')
#train = train.dropna()

In [0]:
#agg_func = lambda d: [(s,w,t) for s,w, t in zip(d["Sent_ID"].values.tolist(),d['Word'].values.tolist(),d["tag"].values.tolist())]

agg_func = lambda d: [(w, t) for w, t in zip(d['Word'].values.tolist(),d["tag"].values.tolist())]
grouped_sents = train.groupby(['Doc_ID','Sent_ID']).apply(agg_func)
sentences = [s for s in grouped_sents]

In [5]:
lens  = [len(s) for s in sentences]    
maxlen = max(lens)
print ('max', maxlen)
print(lens)
print(len(sentences))

max 3899
[15, 39, 25, 39, 15, 17, 18, 17, 26, 2, 13, 53, 16, 32, 23, 23, 9, 30, 34, 23, 16, 14, 19, 11, 12, 28, 9, 26, 10, 27, 26, 31, 24, 31, 19, 26, 31, 5, 14, 20, 12, 24, 14, 14, 17, 39, 33, 19, 25, 12, 21, 20, 33, 17, 38, 23, 27, 15, 7, 15, 16, 12, 10, 16, 17, 15, 41, 35, 28, 18, 11, 31, 33, 14, 41, 5, 9, 20, 29, 18, 18, 18, 14, 13, 25, 39, 16, 5, 5, 18, 25, 21, 49, 17, 35, 14, 18, 21, 14, 24, 12, 28, 16, 17, 13, 21, 16, 35, 34, 53, 45, 18, 27, 3, 4, 24, 11, 18, 9, 21, 12, 41, 16, 31, 26, 32, 9, 9, 12, 16, 17, 36, 78, 48, 12, 62, 49, 42, 53, 38, 26, 25, 17, 37, 37, 27, 15, 31, 7, 55, 8, 23, 34, 49, 35, 34, 27, 17, 29, 22, 5, 9, 20, 35, 22, 28, 17, 19, 9, 23, 24, 13, 8, 5, 27, 20, 29, 10, 21, 30, 12, 23, 31, 27, 25, 26, 30, 19, 14, 18, 19, 19, 44, 34, 28, 31, 21, 15, 24, 10, 20, 39, 32, 31, 46, 30, 15, 21, 6, 86, 19, 48, 14, 31, 37, 17, 51, 28, 22, 17, 27, 18, 22, 13, 28, 38, 21, 22, 18, 28, 14, 29, 24, 34, 44, 15, 26, 13, 29, 26, 9, 13, 15, 29, 16, 34, 20, 10, 12, 8, 18, 13, 13, 16

In [6]:
maxlen = 300
for i in sentences:
  if(len(i)>maxlen):
    lsts = chunks(i,maxlen)
    sentences.remove(i)
    sentences.extend(lsts)
print(len(sentences))

191293


In [7]:
words = list(set(train_copy["Word"].values))
words.append("ENDPAD")
n_words = len(words); print(n_words)
tags = list(set(train_copy["tag"].values))
n_tags = len(tags); print(n_tags)

184507
3


In [0]:
word2idx = {w: i for i, w in enumerate(words)}
tag2idx = {t: i for i, t in enumerate(tags)}

In [0]:
X = [[word2idx[w[0]] for w in s] for s in sentences]
X = pad_sequences(maxlen=300, sequences=X, padding="post",value=n_words - 1)

In [0]:
y = [[tag2idx[w[1]] for w in s] for s in sentences]
y = pad_sequences(maxlen=300, sequences=y, padding="post", value=tag2idx["O"])

In [0]:
from keras.utils import to_categorical
y = [to_categorical(i, num_classes=n_tags) for i in y]

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [0]:
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional

In [14]:
input = Input(shape=(300,))
model = Embedding(input_dim=n_words, output_dim=300, input_length=300)(input)
model = Dropout(0.1)(model)
model = Bidirectional(LSTM(units=100, return_sequences=True, recurrent_dropout=0.1))(model)
out = TimeDistributed(Dense(n_tags, activation="softmax"))(model)  # softmax output layer

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
model = Model(input, out)
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [0]:
history = model.fit(X_train, np.array(y_train), batch_size=32, epochs=1, validation_split=0.2, verbose=1)

In [50]:
#print(len(y_train[:32][0][0]))

model.evaluate(X_test,np.array(y_test),verbose=1)

38259/38259 [==============================] - 785s 21ms/step


[0.0023888513826147584, 0.9992512415511001]

In [0]:
model.save('my_model_2.h5')